In [50]:
import numpy as np
import pandas as pd
from firebase import firebase
import csv
import firebase_admin
import google.cloud
from firebase_admin import credentials, firestore

In [2]:
!pip install google


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [52]:


cred = credentials.Certificate("./ServiceAccountKey.json")
app = firebase_admin.initialize_app(cred)



ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [53]:
store = firestore.client()

file_path = "/data"
collection_name = "academy:register"


'''def batch_data(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]'''


'''data = []
headers = []
with open(file_path) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            for header in row:
                headers.append(header)
            line_count += 1
        else:
            obj = {}
            for idx, item in enumerate(row):
                obj[headers[idx]] = item
            data.append(obj)
            line_count += 1
    print(f'Processed {line_count} lines.')
'''


np_file=np.load('dataframe_students_teacher.npz',allow_pickle=True)
x_values=np_file['arr_0']
col_names=np_file['arr_1']
df= pd.DataFrame(x_values,columns=col_names)
df

,NAME,ROLE,FACIAL_FEATURES
0,ANGELINA JOLIE,STUDENT,"[0.43242025, 1.0748498, -0.12862365, -1.833291..."
1,BARACK OBAMA,TEACHER,"[0.9485809, -0.5584051, 0.094820715, -1.279659..."
2,CHRIS EVANS,STUDENT,"[0.7201714, -1.4684148, -0.7781253, -1.4601073..."
3,MORGAN FREEMAN,TEACHER,"[0.48535094, -0.6192554, -0.4069932, 1.0687788..."
4,SCARLETT JOHANSSON,STUDENT,"[0.19131956, -0.48843804, -1.690233, 0.4423967..."


In [54]:

df['NAME_ROLE']=df['NAME']+'@'+df['ROLE']
df


,NAME,ROLE,FACIAL_FEATURES,NAME_ROLE
0,ANGELINA JOLIE,STUDENT,"[0.43242025, 1.0748498, -0.12862365, -1.833291...",ANGELINA JOLIE@STUDENT
1,BARACK OBAMA,TEACHER,"[0.9485809, -0.5584051, 0.094820715, -1.279659...",BARACK OBAMA@TEACHER
2,CHRIS EVANS,STUDENT,"[0.7201714, -1.4684148, -0.7781253, -1.4601073...",CHRIS EVANS@STUDENT
3,MORGAN FREEMAN,TEACHER,"[0.48535094, -0.6192554, -0.4069932, 1.0687788...",MORGAN FREEMAN@TEACHER
4,SCARLETT JOHANSSON,STUDENT,"[0.19131956, -0.48843804, -1.690233, 0.4423967...",SCARLETT JOHANSSON@STUDENT


In [55]:
records=df[['NAME_ROLE','FACIAL_FEATURES']].to_dict(orient='records')
len(records)

5

In [36]:
# SAVE DTATA IN FIREBASE

In [56]:
data={}
for record in records:
    name_role=record['NAME_ROLE']
    vector=record['FACIAL_FEATURES']
    vector_bytes=vector.tobytes()
    data[name_role]=vector_bytes
    batch=store.batch()
    doc_ref=store.collection(collection_name).document('facial_features')
    batch.set(doc_ref,data)
batch.commit()

[update_time {
   seconds: 1744270894
   nanos: 461519000
 }]

In [68]:
import cv2
from insightface.app import FaceAnalysis

In [69]:
faceapp=FaceAnalysis(name='buffalo_sc',root='insightface_model',providers=['CPUExecutionProvider'])
faceapp.prepare(ctx_id=0,det_size=(640,640),det_thresh=0.5)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_sc\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_sc\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [70]:
person_name=input('ENTER YOUR NAME')
trials=3
for i in range(1,trials+1):
   
    role=input("""
    PLEASE CHOOSE
    1.Student
    2. Teacher
    ENTER NUMBER EITHER 1 or 2"""
              )
    if role in ('1','2'):
        if role=='1':
            role='STUDENT'
        else:
            role='TEACHER'
        break
    else:
        print('INVALID TRY AGAIN')
        if i==3:
            print('EXCEEDS MAX TRIALS')
key=person_name+'@'+role
print('YOUR NAME ',person_name)
print('YOUR ROLE ',role)
print('key=',key)
name_role=key

ENTER YOUR NAME Srinivas

    PLEASE CHOOSE
    1.Student
    2. Teacher
    ENTER NUMBER EITHER 1 or 2 1


YOUR NAME  Srinivas
YOUR ROLE  STUDENT
key= Srinivas@STUDENT


In [41]:
'''for batched_data in batch_data(data, 499):
    batch = store.batch()
    for data_item in batched_data:
        doc_ref = store.collection(collection_name).document()
        batch.set(doc_ref, data_item)
    batch.commit()

print('Done')'''

"for batched_data in batch_data(data, 499):\n    batch = store.batch()\n    for data_item in batched_data:\n        doc_ref = store.collection(collection_name).document()\n        batch.set(doc_ref, data_item)\n    batch.commit()\n\nprint('Done')"

In [71]:
cap=cv2.VideoCapture(0)
sample=0
face_embeddings=[]
while True:
    ret,frame=cap.read()
    if ret==False:
        print("UNABLE TO READ CAMERA")
        break
    results=faceapp.get(frame,max_num=1)
    for res in results:
        sample+=1
        x1,y1,x2,y2=res['bbox'].astype(int)
        cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),1)
        embeddings=res['embedding']
        face_embeddings.append(embeddings)
        if sample>=200:
            break
    cv2.imshow('frame',frame)
    if cv2.waitKey(1)==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [72]:
face_embeddings

[array([-1.62796885e-01,  5.12987375e-02, -1.57545447e-01,  1.77094221e+00,
        -4.53917921e-01,  4.34680164e-01, -1.54704660e-01, -6.99263334e-01,
        -8.40763211e-01, -4.91786838e-01,  4.81317312e-01, -1.37432683e+00,
        -7.37578392e-01,  2.36117840e-01,  5.13556004e-01,  1.11778915e+00,
        -9.64657366e-02, -1.42831236e-01, -3.26801687e-01,  1.72632539e+00,
        -3.19118023e-01,  1.46953309e+00, -1.96853817e-01,  1.39051497e+00,
         8.68407845e-01,  3.21166277e-01, -2.14209318e+00,  1.74974883e+00,
         2.52267051e+00,  3.80180508e-01, -5.09190083e-01, -1.09814787e+00,
        -5.35085559e-01, -1.33229339e+00,  7.28474259e-01,  4.50927973e-01,
         1.10998821e+00,  6.85897291e-01, -4.70441818e-01, -9.75483298e-01,
         1.11075938e+00,  1.03455603e+00,  1.73567891e-01, -3.02873421e+00,
         3.81450653e-02,  2.01276571e-01, -9.48548794e-01, -5.40895820e-01,
         1.23405445e+00,  1.75514686e+00, -6.05474472e-01,  5.88209033e-02,
         3.4

In [73]:
len(face_embeddings)

358

In [74]:
if(len(face_embeddings)>0):
    facial_features=np.asarray(face_embeddings).mean(axis=0)
    facial_features.shape
else:
    facial_features=np.array([])
    facial_features.shape

In [75]:
name_role

'Srinivas@STUDENT'

In [76]:
collection_name

'academy:register'

In [77]:
facial_features

array([-4.3492851e-01, -8.4287375e-01,  2.5821286e-01,  1.0254550e+00,
       -3.3305991e-01,  2.9551828e-01,  7.4313141e-02, -3.1076306e-01,
       -4.2502910e-01, -4.0168869e-01, -9.9017024e-02, -1.3166301e+00,
       -1.1427938e+00, -2.3058690e-01,  2.0102447e-01,  1.0999895e+00,
        1.9200443e-01, -1.2503974e+00, -2.7109846e-01,  1.6867557e+00,
       -1.3551600e-01,  1.3209434e+00, -1.5032227e-01,  7.8676993e-01,
        2.7623844e-01,  3.2868671e-01, -2.1156769e+00,  2.0220544e+00,
        1.9861041e+00, -1.1163152e-01,  1.3972963e-01, -1.2563494e+00,
       -4.3548952e-04, -1.0906578e+00,  8.8259953e-01, -3.3233878e-01,
        1.0466561e+00,  8.5366607e-02, -8.7092471e-01, -8.7700886e-01,
        9.2436284e-02,  6.9335514e-01,  3.0703127e-01, -2.4028552e+00,
        6.2144667e-02,  4.2294401e-01, -6.8883407e-01, -8.2925864e-02,
        1.7681004e+00,  1.3775804e+00,  6.7240930e-01,  7.1531668e-02,
        2.2322392e+00,  1.3696895e+00,  6.7243195e-01, -2.5627375e-01,
      

In [56]:
'''doc_ref = store.collection(collection_name).document('facial_features')

# Convert to bytes
facial_features_bytes = facial_features.tobytes()

# Step 1: Get existing data
doc = doc_ref.get()
if doc.exists:
    existing_data = doc.to_dict()
else:
    existing_data = {}

# Step 2: Get or create list for this person
features_list = existing_data.get(name_role, [])
features_list.append(facial_features_bytes)

# Step 3: Update only the field (not the whole doc)
doc_ref.update({
    name_role: features_list
})'''

update_time {
  seconds: 1744255925
  nanos: 77445000
}

In [49]:
'''doc_ref = store.collection(collection_name).document('facial_features')
facial_features_bytes=facial_features.tobytes()
update_data={name_role:facial_features_bytes}
doc_ref.set(update_data,merge=True)'''

update_time {
  seconds: 1744270429
  nanos: 257973000
}

In [78]:
doc_ref = store.collection(collection_name).document('facial_features')
facial_features_bytes=facial_features.tobytes()
# 1. Get existing data
doc = doc_ref.get()
if doc.exists:
    data = doc.to_dict()
    # 2. Check if this name_role already exists
    if name_role not in data:
        # 3. Only add if it's new
        doc_ref.set({name_role: facial_features_bytes}, merge=True)
        print("✅ Face data added.")
    else:
        print("⚠️ Face data for this user already exists. Not overwriting.")
else:
    # First time creation
    doc_ref.set({name_role: encoded_features})
    print("🆕 Document created with first face data.")

✅ Face data added.


In [47]:
'''doc_ref = store.collection(collection_name).document('facial_features')

# Get existing data
doc = doc_ref.get()

doc'''

In [38]:
'''doc_ref = store.collection(collection_name).document('facial_features')

# Convert facial features to bytes
facial_features_bytes = facial_features.tobytes()

# Append to the existing list for `name_role` using ArrayUnion
doc_ref.update({
    name_role: firestore.ArrayUnion([facial_features_bytes])
})'''

update_time {
  seconds: 1744240854
  nanos: 498745000
}
transform_results {
  null_value: NULL_VALUE
}

In [17]:
'''doc_ref = store.collection(collection_name).document('facial_features')

# Get existing data
doc = doc_ref.get()
if doc.exists:
    existing_data = doc.to_dict()
else:
    existing_data = {}

# Add new data to existing data
existing_data[name_role] = facial_features.tobytes()

# Update the document
doc_ref.set(existing_data)'''

update_time {
  seconds: 1744240195
  nanos: 43874000
}

In [18]:
'''facial_features_bytes=facial_features.tobytes()
data[name_role]=facial_features_bytes

doc_ref=store.collection(collection_name).document('facial_features').update({name_role:facial_features_bytes})'''


"facial_features_bytes=facial_features.tobytes()\ndata[name_role]=facial_features_bytes\n\ndoc_ref=store.collection(collection_name).document('facial_features').update({name_role:facial_features_bytes})"

In [21]:
import numpy as np
x_mean=np.asarray(face_embeddings).mean(axis=0)
x_mean.shape

(512,)

In [22]:
x_mean_bytes=x_mean.tobytes()

In [23]:
retrive_data=store.collection(collection_name).document('facial_features').get()
retrive_data
retrive_face=retrive_data.to_dict()
retrive_face

{'SCARLETT JOHANSSON@STUDENT': b'F\xe9C>\x8d\x14\xfa\xbe\x8eY\xd8\xbf\xd3\x81\xe2>O\x03F?\xc3T\xf5\xbe\xf9\xfb\x16\xbfmX\xaa?\xf8\xf9\x96\xbf\xc89\x8f\xbf\x00\xc4\xbc\xba\xde\x194\xbe\x86\xd1\x1b>\xe0\x7fS\xbe\x9d\x8f\xdf\xbe\xc6)\xc7>\x1c\xde!>:\x9fr\xbe\xbd\xe9\xf5>@\x1a*\xc0\xc6?\x9b\xbf\xa4^\x0e\xc0\xc1\n\x8b?\xe5\x9d\x1d>\'\x14(?O\x8c*\xbe\xda\xdf\\\xbeXm\x1f\xbc\x03a\xb2\xbf:{\x9c?r\xb6\x87\xbf\xa6P\x14?*P\xbb>\x10{\xe7\xbf\x92\x06A\xbe\x1b6\xec\xbd8\x08\x1f\xbf0\x9c\xd3>\xc6\x19\xad=\x8d\xf7#\xbf*\xa5 >\x88+\x9a?5\xe79>\r\xdd\x91\xbd\xbd\xff\r\xc0\xb7Z\x99?*\xbdc\xbd\xf6\xa7e\xbf)\xd8\xb7\xbe|L\x92\xbf\x16\x83E\xbe\x1av\t?\x8a<\x9d>.\xb14\xbf\xdeA\x85\xbf\xfa\xfdG>\x9b\xe5\xb0>\x95c\xca>*\xeek\xbdVL\xcd\xbf\x8eU\xae?\xeaN\x15>f\x1b\x1c@\x12\x9c\x04\xbe\xd9\xab\x94\xbf\xb5\xd4\x13?p\xd3z\xbf2\x99\x89\xbf\xcaw\xb9\xbew\x87/?P}\xdf>+\x8c\xc6\xbf\x0e\x91<=\xa6z\xbe\xbf.\x93\xe8\xbej\\\xe6\xbe(\rt?\xcaC\x13>\x846\xbb>\x86\x9d\xb5?\xfb{\xe2?CO+?\xabA1\xbf\x01]\xc0?\xde\xee\xab\xbf\xe3

In [64]:
def decode_facial_data(val):
    if isinstance(val, bytes):
        return [np.frombuffer(val, dtype=np.float32)]
    elif isinstance(val, list):
        return [np.frombuffer(f, dtype=np.float32) for f in val if isinstance(f, bytes)]
    else:
        return []  # fallback in case of unexpected types

retrive_series = pd.Series(retrive_face)
retrive_series = retrive_series.apply(decode_facial_data)


In [24]:
retrive_series=pd.Series(retrive_face)
retrive_series

SCARLETT JOHANSSON@STUDENT    b'F\xe9C>\x8d\x14\xfa\xbe\x8eY\xd8\xbf\xd3\x81...
MORGAN FREEMAN@TEACHER        b'\xeb\x7f\xf8>\x86\x87\x1e\xbfja\xd0\xbe\xbe\...
CHRIS EVANS@STUDENT           b'\']8?\x04\xf5\xbb\xbf83G\xbf\xcc\xe4\xba\xbf...
ANGELINA JOLIE@STUDENT        b'0f\xdd>\xae\x94\x89?\xeb\xb5\x03\xbeN\xa9\xe...
BARACK OBAMA@TEACHER          b'3\xd6r?\xa3\xf3\x0e\xbf]1\xc2=\xde\xcb\xa3\x...
Yuthika@STUDENT               b'Z\x92p\xbfW\x0b\x9c\xbe\xe9\x96\x89>\xfa4\xa...
dtype: object

In [63]:
'''retrive_series = retrive_series.apply(
    lambda lst: [np.frombuffer(f, dtype=np.float32) for f in lst]
)'''


TypeError: a bytes-like object is required, not 'int'

In [61]:
'''retrive_series = retrive_series.apply(lambda x: np.frombuffer(x, dtype=np.float32))
index = retrive_series.index.tolist()  # Extract index as a list of strings
'''

TypeError: a bytes-like object is required, not 'list'

In [25]:
retrive_df=retrive_series.to_frame().reset_index()
retrive_df.columns=['NAME_ROLE','FACIAL_FEATURES']
retrive_df

,NAME_ROLE,FACIAL_FEATURES
0,SCARLETT JOHANSSON@STUDENT,b'F\xe9C>\x8d\x14\xfa\xbe\x8eY\xd8\xbf\xd3\x81...
1,MORGAN FREEMAN@TEACHER,b'\xeb\x7f\xf8>\x86\x87\x1e\xbfja\xd0\xbe\xbe\...
2,CHRIS EVANS@STUDENT,b'\']8?\x04\xf5\xbb\xbf83G\xbf\xcc\xe4\xba\xbf...
3,ANGELINA JOLIE@STUDENT,b'0f\xdd>\xae\x94\x89?\xeb\xb5\x03\xbeN\xa9\xe...
4,BARACK OBAMA@TEACHER,b'3\xd6r?\xa3\xf3\x0e\xbf]1\xc2=\xde\xcb\xa3\x...
5,Yuthika@STUDENT,b'Z\x92p\xbfW\x0b\x9c\xbe\xe9\x96\x89>\xfa4\xa...
